In [112]:
#first step is to load the packages that I will need.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
import xlrd


In [113]:
#first step is to load the 2 tabs of data with the revelant info from my Excel Spreadsheet.

xls = pd.ExcelFile('egrid2018_data_v2.xlsx')



df1 = pd.read_excel(xls, 'UNT18')
df2 = pd.read_excel(xls, 'GEN18')

In [114]:
xls.sheet_names  # see all sheet names

['Contents',
 'Idea',
 'UNT18',
 'GEN18',
 'PLNT18',
 'ST18',
 'BA18',
 'SRL18',
 'NRL18',
 'US18',
 'GGL18']

In [115]:
print(df1.columns)

print(df2.columns)

Index(['eGRID 2018 Unit file sequence number', 'Data Year',
       'Plant state abbreviation', 'Plant name',
       'DOE/EIA ORIS plant or facility code', 'Unit ID', 'Prime Mover',
       'Unit Operational Status', 'CAMD Program flag', 'Program code(s)',
       'Unit bottom and firing type', 'Number of associated generators',
       'Unit primary fuel', 'Unit operating hours',
       'Unit unadjusted annual heat input (MMBtu)',
       'Unit unadjusted ozone season heat input (MMBtu)',
       'Unit unadjusted annual NOx emissions (tons)',
       'Unit unadjusted ozone season NOx emissions (tons)',
       'Unit unadjusted annual SO2 emissions (tons)',
       'Unit unadjusted annual CO2 emissions (tons)',
       'Unit unadjusted annual Hg emissions (lbs)',
       'Unit unadjusted annual heat input source',
       'Unit unadjusted ozone season heat input source',
       'Unit unadjusted annual NOx emissions source',
       'Unit unadjusted ozone season NOx emissions source',
       'Unit u

In [116]:
#print(df1.describe())

print(df1['Data Year'].value_counts())

2018    26732
YEAR        1
Name: Data Year, dtype: int64


In [117]:
print(df1.iloc[0,:])
print(df1.head())

eGRID 2018 Unit file sequence number                 SEQUNT18
Data Year                                                YEAR
Plant state abbreviation                             PSTATABB
Plant name                                              PNAME
DOE/EIA ORIS plant or facility code                    ORISPL
Unit ID                                                UNITID
Prime Mover                                             PRMVR
Unit Operational Status                               UNTOPST
CAMD Program flag                                    CAMDFLAG
Program code(s)                                       PRGCODE
Unit bottom and firing type                          BOTFIRTY
Number of associated generators                        NUMGEN
Unit primary fuel                                      FUELU1
Unit operating hours                                    HRSOP
Unit unadjusted annual heat input (MMBtu)               HTIAN
Unit unadjusted ozone season heat input (MMBtu)         HTIOZ
Unit una

In [118]:
# i want to get rid of this extra row up top in both dfs
df1 = df1.drop(df1.index[0])

In [119]:
df2 = df2.drop(df2.index[0])

In [120]:
# I need to combine these datasets, but there is no matching unique identifier I can use to map them.

df1["Plant name"].value_counts()
df1["Plant name"].count()





26732

In [121]:
df2["Plant name"].value_counts()
df2["Plant name"].count()

27944

In [122]:
df1['Plant name'].nunique()

10933

In [123]:
df2['Plant name'].nunique()

10947

In [124]:
#How can i generate a unique value in both tables for each row so that I can merge them on that value?  I can take Plant Name + 1, Plant Name + 2, etc.
#They don't line up correctly and probably aren't meant to be viewed that way, luckily there is another tab for Plant Summary stats that has all of the values together. I'm going to use that. 

df3 = pd.read_excel(xls, 'PLNT18')


In [125]:
print(df3.columns)
print(df3.iloc[0,:])
print(df3.head())

Index(['eGRID2018 Plant file sequence number', 'Data Year',
       'Plant state abbreviation', 'Plant name',
       'DOE/EIA ORIS plant or facility code',
       'Plant transmission or distribution system owner name',
       'Plant transmission or distribution system owner ID', 'Utility name',
       'Utility ID', 'Plant-level sector',
       ...
       'Plant wind generation percent (resource mix)',
       'Plant solar generation percent (resource mix)',
       'Plant geothermal generation percent (resource mix)',
       'Plant other fossil generation percent (resource mix)',
       'Plant other unknown / purchased fuel generation percent (resource mix)',
       'Plant total nonrenewables generation percent (resource mix)',
       'Plant total renewables generation percent (resource mix)',
       'Plant total nonhydro renewables generation percent (resource mix)',
       'Plant total combustion generation percent (resource mix)',
       'Plant total noncombustion generation percent (r

In [126]:
# i want to get rid of this extra row up top in both dfs
df3 = df3.drop(df1.index[0])

In [127]:
print(df3.head())

  eGRID2018 Plant file sequence number Data Year Plant state abbreviation  \
0                             SEQPLT18      YEAR                 PSTATABB   
2                                    2      2018                       AK   
3                                    3      2018                       AK   
4                                    4      2018                       AK   
5                                    5      2018                       AK   

                         Plant name DOE/EIA ORIS plant or facility code  \
0                             PNAME                              ORISPL   
2  Agrium Kenai Nitrogen Operations                               54452   
3                          Alakanuk                               57053   
4               Allison Creek Hydro                               58982   
5                            Ambler                               60243   

  Plant transmission or distribution system owner name  \
0                           

In [128]:
columns = df3.columns.to_list()

In [129]:
print(columns)

#I can get rid of a lot of these columns.



['eGRID2018 Plant file sequence number', 'Data Year', 'Plant state abbreviation', 'Plant name', 'DOE/EIA ORIS plant or facility code', 'Plant transmission or distribution system owner name', 'Plant transmission or distribution system owner ID', 'Utility name', 'Utility ID', 'Plant-level sector', 'Balancing Authority Name', 'Balancing Authority Code', 'NERC region acronym', 'eGRID subregion acronym', 'eGRID subregion name', 'Plant associated ISO/RTO Territory ', 'Plant FIPS state code', 'Plant FIPS county code', 'Plant county name', 'Plant latitude', 'Plant longitude', 'Number of units', 'Number of generators', 'Plant primary fuel', 'Plant primary coal/oil/gas/ other fossil fuel category', 'Flag indicating if the plant burned or generated any amount of coal', 'Plant capacity factor', 'Plant nameplate capacity (MW)', 'Nonbaseload Factor', 'Biogas/ biomass plant adjustment flag', 'Combined heat and power (CHP) plant adjustment flag', 'CHP plant useful thermal output (MMBtu)', 'CHP plant p

In [130]:
# I will just make a new df of the columsn I care about

columns_of_interest = ['Data Year', 'Plant state abbreviation', 'Utility name', 'Plant name', 'Plant-level sector', 'Balancing Authority Name', 'eGRID subregion name', 'Plant latitude', 'Plant longitude', 'Number of units', 'Number of generators', 'Plant primary coal/oil/gas/ other fossil fuel category','Flag indicating if the plant burned or generated any amount of coal', 'Plant capacity factor', 'Plant nameplate capacity (MW)', 'Nonbaseload Factor', 'Plant annual net generation (MWh)', 'Plant annual CO2 emissions (tons)', 'Plant annual CH4 emissions (lbs)', 'Plant annual CO2 total output emission rate (lb/MWh)', 'Plant annual CH4 total output emission rate (lb/MWh)', 'Plant annual total nonrenewables net generation (MWh)', 'Plant annual total renewables net generation (MWh)', 'Plant total nonrenewables generation percent (resource mix)', 'Plant total renewables generation percent (resource mix)']




In [133]:
df3 = df3[columns_of_interest]


print(df3.head())

   index Data Year Plant state abbreviation                   Utility name  \
0      0      YEAR                 PSTATABB                       UTLSRVNM   
1      2      2018                       AK                  Agrium US Inc   
2      3      2018                       AK  Alaska Village Elec Coop, Inc   
3      4      2018                       AK   Copper Valley Elec Assn, Inc   
4      5      2018                       AK  Alaska Village Elec Coop, Inc   

                         Plant name Plant-level sector  \
0                             PNAME             SECTOR   
1  Agrium Kenai Nitrogen Operations     Industrial CHP   
2                          Alakanuk   Electric Utility   
3               Allison Creek Hydro   Electric Utility   
4                            Ambler   Electric Utility   

  Balancing Authority Name eGRID subregion name Plant latitude  \
0                   BANAME               SRNAME            LAT   
1     Alaska Miscellaneous     ASCC Alaska Grid   